# Homework 1

## Dataset 1: API

### Gathering the Data

I was first interested in using an API to obtain data, having previous expereince working with data in text and CSV formatts. From the [list](https://github.com/toddmotto/public-apis) of public APIs listed by Github user 'toddmotto', I selected the [iex](https://iextrading.com/developer/docs/#quote) financial data API. I then leveraged the [code](https://github.com/himoacs/iex_data/blob/master/iex_data.py) written by github user 'himoacs' to retrieve fiancial data about 10 technology companies. 

The cell below contains the code to create the class 'API'.

In [ ]:
import pandas as pd
from urllib.request import Request, urlopen
import json
from pandas.io.json import json_normalize

In [8]:
class API(object):

    """
    This API class allows users to get different type of data from IEX via its methods.
    API class can be used for:
    - requesting valid securities
    - getting latest quote and trade data
    - getting latest trade data
    Examples:
        m = API()
        print(m.get_latest_trade_data(['AAPL', 'IBM']))
        print(m.get_latest_quote_and_trade_data(['AAPL', 'IBM']))
        print(m.return_valid_securities(['AAPL', 'IBM']))
    """

    def __init__(self):
        self._end_point_prefix = r'https://api.iextrading.com/1.0/'

    def return_valid_securities(self, securities):

        """
        Return a list of valid securities
        :param securities: list of securities
        :return: list of valid securities
        """

        suffix = r'ref-data/symbols'
        valid_securities = self._url_to_dataframe(self._end_point_prefix+suffix)['symbol']

        return [x for x in securities if x in set(valid_securities)]

    def _url_to_dataframe(self, url, nest=None):

        """
        Takes a url and returns the response in a pandas dataframe
        :param url: str url
        :param nest: column with nested data
        :return: pandas dataframe containing data from url
        """

        request = Request(url)
        response = urlopen(request)
        elevations = response.read()
        data = json.loads(elevations)

        if nest:
            data = json_normalize(data[nest])
        else:
            data = json_normalize(data)

        return pd.DataFrame(data)

    def get_latest_quote_and_trade(self, securities):

        """
        Gets latest quote and trade data
        :param securities: list of securities
        :return: pandas dataframe containing data for valid securities
        """

        securities = self.return_valid_securities(securities)

        if securities:
            suffix = r'tops?symbols='
            symbols = ','.join(securities)
            df = self._url_to_dataframe(self._end_point_prefix + suffix + symbols)
            df['lastSaleTime'] = pd.to_datetime(df['lastSaleTime'], unit='ms')
            df['lastUpdated'] = pd.to_datetime(df['lastUpdated'], unit='ms')
            df.set_index(['symbol'], inplace=True)
            return df
        else:
            print('These stock(s) are invalid!')

    def get_latest_trade(self, securities):

        """
        Gets latest trade data
        :param securities: list of securities
        :return: pandas dataframe containing data for valid securities
        """

        securities = self.return_valid_securities(securities)

        if securities:
            suffix = r'tops/last?symbols='
            symbols = ','.join(securities)
            df = self._url_to_dataframe(self._end_point_prefix + suffix + symbols)
            df['time'] = pd.to_datetime(df['time'], unit='ms')
            df.set_index(['symbol'], inplace=True)
            return df
        else:
            print('These stock(s) are invalid!')

    def get_latest_news(self, securities, count=1):

        """
        Get latest news for securities. By default, gets one news item per security.
        :param securities: list of securities
        :param count: how many news items to return
        :return: dataframe
        """

        securities = self.return_valid_securities(securities)

        final_df = pd.DataFrame({})

        # Get news for each security and then append the results together
        if securities:
            for symbol in securities:
                suffix = r'stock/{symbol}/news/last/{count}'.format(symbol=symbol,
                                                                    count=count)
                df = self._url_to_dataframe(self._end_point_prefix + suffix)
                df['time'] = pd.to_datetime(df['datetime'])
                del df['datetime']
                df['symbol'] = symbol
                df = df[['symbol', 'time', 'headline', 'summary', 'source', 'url', 'related']]
                final_df = final_df.append(df, ignore_index=True)
            return final_df
        else:
            print('These stock(s) are invalid!')

    def get_financials(self, securities):

        """
        Get latest financials of securities. By default, gets one news item per security.
        :param securities: list of symbols
        :return: dataframe
        """

        securities = self.return_valid_securities(securities)

        final_df = pd.DataFrame({})

        # Get financials of each security and then append the results together
        if securities:
            for symbol in securities:
                suffix = r'stock/{symbol}/financials'.format(symbol=symbol)
                df = self._url_to_dataframe(self._end_point_prefix + suffix, 'financials')
                df['symbol'] = symbol
                final_df = final_df.append(df, ignore_index=True)
            return final_df
        else:
            print('These stock(s) are invalid!')

    def get_earnings(self, securities):

        """
        Get latest earnings for securities.
        :param securities: list of symbols
        :return: dataframe
        """

        securities = self.return_valid_securities(securities)

        final_df = pd.DataFrame({})

        # Get earnings data for each security and then append the results together
        if securities:
            for symbol in securities:
                suffix = r'stock/{symbol}/earnings'.format(symbol=symbol)
                df = self._url_to_dataframe(self._end_point_prefix + suffix, 'earnings')
                df['symbol'] = symbol
                final_df = final_df.append(df, ignore_index=True)
            return final_df
        else:
            print('These stock(s) are invalid!')

    def get_trade_bars_data(self, securities, bucket='1m'):

        """
        Get bucketed trade/volume data. Supported buckets are: 1m, 3m, 6m, 1y, ytd, 2y, 5y
        :param securities: list of securities
        :param bucket:
        :return: dataframe
        """

        securities = self.return_valid_securities(securities)

        final_df = pd.DataFrame({})

        # Get earnings data for each security and then append the results together
        if securities:
            for symbol in securities:
                suffix = r'stock/{symbol}/chart/{bucket}'.format(symbol=symbol,
                                                                   bucket=bucket)
                df = self._url_to_dataframe(self._end_point_prefix + suffix)
                df['symbol'] = symbol
                final_df = final_df.append(df, ignore_index=True)
            return final_df
        else:
            print('These stock(s) are invalid!')

I used the get_financials() function to obtain four quarters worth of financial data about the first ten technology companies to come to mind. The companies, in order, were:
- Apple 
- Google
- Facebook
- Amazon
- Microsoft
- Intel
- IBM
- HP
- Cisco Systems
- Oracle

In [2]:
m = API()
ticker_list = ['AAPL', 'GOOGL', 'FB', 'AMZN', 'MSFT', 'INTC', 'IBM', 'HPE', 'CSCO', 'ORCL']
fin_df = pd.DataFrame(m.get_financials(ticker_list))

As you can see below, the data consists of accounting measures tpyically reported by publicly-traded companies. For exampe: revenue, assets, liabilities, operating expesne, gross profit, and net income.  

In [3]:
fin_df[fin_df['symbol']=='AAPL']

,cashChange,cashFlow,costOfRevenue,currentAssets,currentCash,currentDebt,grossProfit,netIncome,operatingExpense,operatingGainsLosses,...,operatingRevenue,reportDate,researchAndDevelopment,shareholderEquity,totalAssets,totalCash,totalDebt,totalLiabilities,totalRevenue,symbol
0,1718000000,15656000000,32648000000,128645000000,20289000000,18473000000,19931000000,10714000000,6811000000,None,...,52579000000,2017-09-30,2997000000,134047000000,375319000000,74181000000,1.1568e+11,241272000000,52579000000,AAPL
1,3414000000,8363000000,27920000000,112875000000,18571000000,18475000000,17488000000,8717000000,6720000000,None,...,45408000000,2017-06-30,2937000000,132425000000,345173000000,76759000000,1.08339e+11,212748000000,45408000000,AAPL
2,-1214000000,12523000000,32305000000,101990000000,15157000000,13991000000,20591000000,11029000000,6494000000,None,...,52896000000,2017-03-31,2776000000,134082000000,334532000000,67101000000,9.8522e+10,200450000000,52896000000,AAPL
3,-4113000000,27056000000,48175000000,103332000000,16371000000,13992000000,30176000000,17891000000,6817000000,None,...,78351000000,2016-12-31,2871000000,132390000000,331141000000,60452000000,8.7549e+10,198751000000,78351000000,AAPL


### Cleaning the Data

I first converted the 'reportDate' variable from a non-null object to a non-nul datetime64 object.

In [4]:
fin_df['reportDate'] = pd.to_datetime(fin_df['reportDate'])
fin_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 21 columns):
cashChange                40 non-null int64
cashFlow                  40 non-null int64
costOfRevenue             40 non-null int64
currentAssets             40 non-null int64
currentCash               40 non-null int64
currentDebt               28 non-null object
grossProfit               40 non-null int64
netIncome                 40 non-null int64
operatingExpense          40 non-null int64
operatingGainsLosses      24 non-null object
operatingIncome           40 non-null int64
operatingRevenue          40 non-null int64
reportDate                40 non-null datetime64[ns]
researchAndDevelopment    40 non-null int64
shareholderEquity         40 non-null int64
totalAssets               40 non-null int64
totalCash                 40 non-null int64
totalDebt                 29 non-null object
totalLiabilities          40 non-null int64
totalRevenue              40 non-null int64
symbo

### Analyzing the Data

In [5]:
fin_df.corr()

,cashChange,cashFlow,costOfRevenue,currentAssets,currentCash,grossProfit,netIncome,operatingExpense,operatingIncome,operatingRevenue,researchAndDevelopment,shareholderEquity,totalAssets,totalCash,totalLiabilities,totalRevenue
cashChange,1.000000,-0.018201,-0.102972,-0.095501,0.275348,-0.144052,-0.125989,-0.075371,-0.128280,-0.122230,-0.064191,-0.011465,-0.070554,-0.085615,-0.088071,-0.122484
cashFlow,-0.018201,1.000000,0.616775,0.606915,0.185903,0.847846,0.894821,0.265855,0.889380,0.728579,0.372436,0.666254,0.765012,0.483841,0.579945,0.728263
costOfRevenue,-0.102972,0.616775,1.000000,0.232165,0.384039,0.808830,0.582855,0.558554,0.617961,0.977161,0.490214,0.284146,0.610422,0.025344,0.633998,0.977414
currentAssets,-0.095501,0.606915,0.232165,1.000000,0.190960,0.621326,0.596095,0.328686,0.550582,0.383281,0.346504,0.662366,0.790497,0.961550,0.617313,0.382731
currentCash,0.275348,0.185903,0.384039,0.190960,1.000000,0.316754,0.178901,0.241702,0.224650,0.377768,0.157596,0.232858,0.325024,0.160335,0.281139,0.377569
grossProfit,-0.144052,0.847846,0.808830,0.621326,0.316754,1.000000,0.805609,0.630187,0.809661,0.915313,0.673106,0.633935,0.786006,0.469562,0.630876,0.914835
netIncome,-0.125989,0.894821,0.582855,0.596095,0.178901,0.805609,1.000000,0.059426,0.991100,0.690085,0.194194,0.731186,0.853609,0.458524,0.655571,0.689790
operatingExpense,-0.075371,0.265855,0.558554,0.328686,0.241702,0.630187,0.059426,1.000000,0.054545,0.610542,0.890290,0.119717,0.226808,0.251490,0.225783,0.610057
operatingIncome,-0.128280,0.889380,0.617961,0.550582,0.224650,0.809661,0.991100,0.054545,1.000000,0.715602,0.192666,0.724752,0.839369,0.413763,0.640646,0.715354
operatingRevenue,-0.122230,0.728579,0.977161,0.383281,0.377768,0.915313,0.690085,0.610542,0.715602,1.000000,0.579164,0.423323,0.701900,0.186777,0.662168,0.999998


In [6]:
import numpy as np
#pd.pivot_table(train_df, values='hoursperweek', index=['relationship'], columns=['label'], aggfunc=np.mean)
pd.pivot_table(fin_df, values = 'netIncome', index = ['reportDate'], aggfunc=np.mean)

,netIncome
reportDate,
2016-12-31,5.829286e+09
2017-01-31,1.307500e+09
2017-02-28,2.239000e+09
2017-03-31,4.251143e+09
2017-04-30,9.515000e+08
2017-05-31,3.232000e+09
2017-06-30,3.997714e+09
2017-07-31,1.294500e+09
2017-08-31,2.210000e+09


In [7]:
# DATA CLEANING
fin_df['quarter'] = Series

NameError: name 'Series' is not defined

## Dataset 2: CSV

### Gathering the Data

My wife teaches English to children in China online through a platform called VIPKID. Despite the very early mornings (Beijing is 15 hours ahead of Utah), she enjoys her job and is paid fairly well for her efforts. 

The platform provides a record of all the lessons she has taught, which are always 30 minutes in length. For each record, there is data about the lesson time, the lesson id/name, the finish type, if the lesson was booked on short notice, and the amount earned by teaching the lesson. 

I manually compiled 8 months of this information into a Google Spreadsheet and downloaded the data as a CSV file for analysis in Python. 

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
vip_df = pd.read_csv("/Users/erikgregorywebb/Downloads/VIPKID_Raw.csv")
vip_df.head()

In [ ]:
vip_df.info()

### Cleaning the Data

To prepare for analysis, I wanted to ‘clean’ the data and ensure I was using the correct data types for each of the variables. For example, I converted the lesson date stamp variable (originally a series of strings called 'Time') into two seperate datetime objects: Date and Time. 

In [ ]:
# Turn the 'Time' variable (which actually includes both date AND time) into seperate 'Date' and 'Time' variables
vip_df['Date'] = ""
for i in range(0, len(vip_df)):
    vip_df.loc[i, 'Date'] = datetime.strptime(vip_df.loc[i, 'Time'], "%m/%d/%Y %H:%M:%S").date()
    vip_df.loc[i, 'Time'] = datetime.strptime(vip_df.loc[i, 'Time'], "%m/%d/%Y %H:%M:%S").time()

Next, I created additional variables by extracting pieces of information from existing variables. For example, I extracted the name of the month as well as the day of the week from the lesson date stamp. From the lesson ID I created a variable indicating if the lesson was a ‘major course’ or a ‘trail lesson’, the two broadest lesson categories. 

In [ ]:
# Rearrange the columns in the name of order
cols = vip_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
vip_df = vip_df[cols]

In [ ]:
vip_df['Category'] = ""
vip_df['Month'] = ""
vip_df["DOW"] = ""

for i in range(0, len(vip_df)):
    vip_df.loc[i, 'Category'] = vip_df.loc[i, 'Lesson_ID'][:2]
    vip_df.loc[i, 'Month'] = vip_df.loc[i, 'Date'].strftime("%m/%Y")
    vip_df.loc[i, 'DOW'] = vip_df.loc[i, 'Date'].strftime("%w")

In [ ]:
vip_df.head()

### Analyzing the Data

With the cleaning and variable manipulation complete, I began my analysis. Although there are several ways to potentially slice and analyze the data, I determined to answer the following five questions:

1. How much money has my wife earned each month for the past 8 months? (Pivot Table)
2. How has the distribution of lesson finish type changed over time? (Crosstab)
3. How has the distribution of major course and trail lessons changed over time? (Stacked Bar Chart)
4. How has the distribution of short notice lessons changed over time?
5. What is the distribution of lesson counts across day of the week and time of day? (Heatmap)

#### 1. How much money has my wife earned each month for the past 8 months?

In [ ]:
vip_pivot_table = pd.pivot_table(vip_df, values = 'Payment', index = ['Month'], aggfunc = np.sum)
vip_pivot_table

In [ ]:
print(sum(vip_pivot_table['Payment']))

It appears that my wife earned the most during the early summer months and then earned less in the Fall when she began working a second job, causing her to work less hours teaching through the VIPKID platform. The month with the highest earned income was June 2017, with a total payment of $1,422.25. 

To date, she has earned a total of $6,187.25. 

#### 2. How has the distribution of lesson finish type changed over time?

In [ ]:
vip_crosstab = pd.crosstab(vip_df['Month'], vip_df['Finish_Type'])
vip_crosstab

This crosstab shows the count of lessons organized by month and finish type. To make a more accurate comparison of how the distribution of lesson finish types have changed over time, I calculated the value of each cell to be its percentage of the row total.

In [ ]:
vip_crosstab_percentage = vip_crosstab.apply(lambda x: x/x.sum(), axis=1)
vip_crosstab_percentage

This revised crosstab demonsrates that the distribution of lesson finish types remained fairly constant over the 8 month period. 

#### 3. How has the distribution of major course and trail lessons changed over time?

In [ ]:
vip_category = pd.pivot_table(vip_df, values = 'Payment', index = ['Month'], columns = ['Category'], aggfunc = 'count')

In [ ]:
N = len(vip_category)
ind = np.arange(N)
Major = vip_category['MC']
Trial = vip_category['T1']
width = 0.35

p1 = plt.bar(ind, Major, width, color='#d62728')
p2 = plt.bar(ind, Trial, width,
             bottom=Major)

plt.ylabel('Lessons Taught')
plt.xlabel('Month')
plt.title('Major vs. Trial Lessons Over Time')
plt.xticks(ind, ('5/17', '6/17', '7/17', '8/17', '9/17', '10/17', '11/17', '12/17'))
plt.legend((p1[0], p2[0]), ('Major', 'Trial'))
plt.show()

#### 4. How has the distribution of short notice lessons changed over time?

In [ ]:
vip_sn = pd.pivot_table(vip_df, values = 'Payment', index = ['Month'], columns = ['Short_ Notice'], aggfunc = 'count')
vip_sn

In [ ]:
vip_sn_percentage = vip_sn.apply(lambda x: x/x.sum(), axis=1)
vip_sn_percentage

#### 5. What is the distribution of lesson counts across day of the week and time of day?

In [ ]:
vip_heatmap = pd.pivot_table(vip_df, values = 'Payment', index = ['Time'], columns = ['DOW'], aggfunc = 'count')
f, ax = plt.subplots(figsize=(12, 8))
sns.heatmap(vip_heatmap, annot = True, linewidths=.5, ax=ax)
plt.show()

In the heatmap above, the Day of the Week (DOW) axixs begin with 0 (Sunday) and ends with 6 (Saturday). There appears to be a high concentration of lessons taught weekdays during the early hours of morning, between 4:30 and 7:00 am. The most frequest lesson day of the week and time is Friday morning at 6 am. 

Helpful Links:
- https://docs.python.org/2/library/datetime.html
- http://www.pythonforbeginners.com/basics/python-datetime-time-examples
- https://stackoverflow.com/questions/26886653/pandas-create-new-column-based-on-values-from-other-columns
- https://stackoverflow.com/questions/31481803/compute-percentage-for-each-row-in-pandas-dataframe
- https://matplotlib.org/examples/pylab_examples/bar_stacked.html

## Dataset 3: API

### Gathering the Data

In [ ]:
# Import Libraries
import requests
from bs4 import BeautifulSoup
import  pandas as pd

In [ ]:
r = requests.get('https://www.basketball-reference.com/players/a/')

In [ ]:
soup = BeautifulSoup(r.content, "lxml")

In [ ]:
data = ""
for tr in soup.find_all("tr"): 
    record = "" 
    for td in tr.find_all(["th", "td"]): 
        record = record + "," + td.text 
    if len(record)!=0:
        data = data + "\n" + record[1:]
#print(data[1:])

In [ ]:
readable_data = pd.compat.StringIO(data)
bb = pd.read_table(readable_data, sep=";")
print(bb)

In [ ]:
# https://stackoverflow.com/questions/32224363/python-convert-comma-separated-list-to-pandas-dataframe